# Samples

- In stats analysis we use a sample to make statements about the population

- The sample does not look exactly like the population

- Need to learn about how samples behave in order to make statements about the population. -> stats inference